# Olympic Data Preprocessing Pipeline

This notebook transforms raw athlete-level Olympic data into team-level aggregated data suitable for medal prediction models.

## Overview
- **Input**: `athlete_events.csv` - Individual athlete records (271,116 rows)
- **Output**: `teams.csv` - Team-level summary statistics (2,144 rows)
- **Goal**: Create features for predicting Olympic medal counts by country/year

## Step 1: Import Libraries and Load Data

In [22]:
import pandas as pd
import numpy as np

# Display all columns when showing dataframes
pd.set_option('display.max_columns', None)

In [23]:
# Load the raw athlete events dataset
# Each row represents one athlete's participation in one event
athletes = pd.read_csv("athlete_events.csv")

print(f"Dataset shape: {athletes.shape}")
print(f"\nColumns: {list(athletes.columns)}")
athletes.head()

Dataset shape: (271116, 15)

Columns: ['ID', 'Name', 'Sex', 'Age', 'Height', 'Weight', 'Team', 'NOC', 'Games', 'Year', 'Season', 'City', 'Sport', 'Event', 'Medal']


,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


### Data Dictionary

| Column | Description |
|--------|-------------|
| ID | Unique athlete identifier |
| Name | Athlete's full name |
| Sex | M (Male) or F (Female) |
| Age | Athlete's age in years |
| Height | Height in centimeters |
| Weight | Weight in kilograms |
| Team | Country/team name |
| NOC | National Olympic Committee 3-letter code |
| Games | Year and season (e.g., "1992 Summer") |
| Year | Olympic year |
| Season | Summer or Winter |
| City | Host city |
| Sport | Sport category |
| Event | Specific event within the sport |
| Medal | Gold, Silver, Bronze, or NaN (no medal) |

## Step 2: Filter for Summer Olympics Only

We focus on Summer Olympics because:
- More events and athletes compared to Winter Olympics
- More consistent participation across countries
- Different sports/dynamics between Summer and Winter

In [24]:
# Keep only Summer Olympics data
athletes = athletes[athletes["Season"] == "Summer"]

print(f"After filtering for Summer Olympics:")
print(f"Dataset shape: {athletes.shape}")
print(f"Years covered: {athletes['Year'].min()} to {athletes['Year'].max()}")
print(f"Number of countries: {athletes['NOC'].nunique()}")

After filtering for Summer Olympics:
Dataset shape: (222552, 15)
Years covered: 1896 to 2016
Number of countries: 230


## Step 3: Define Team-Level Aggregation Function

This function takes all athletes from a single country in a single year and calculates:
- **Team size** metrics (events participated, number of athletes)
- **Athlete demographics** (average age, height, weight)
- **Performance** (total medals won)

In [25]:
def team_summary(data):
    return pd.Series({
        # Team identification
        'team': data.iloc[0, :]["NOC"],          # 3-letter country code (e.g., "USA")
        'country': data.iloc[-1, :]["Team"],     # Full country name (e.g., "United States")
        'year': data.iloc[0, :]["Year"],         # Olympic year

        # Team composition
        'events': len(data['Event'].unique()),   # Number of unique events participated in
        'athletes': data.shape[0],               # Total number of athlete-event entries

        # Average athlete characteristics
        'age': data["Age"].mean(),               # Mean age of athletes
        'height': data['Height'].mean(),         # Mean height in cm
        'weight': data['Weight'].mean(),         # Mean weight in kg

        # Performance metric
        'medals': sum(~pd.isnull(data["Medal"])) # Count of non-null Medal values
    })

### Understanding the Aggregation

**Example**: USA in 2016
- Had 719 athlete-event entries (some athletes compete in multiple events)
- Competed in 245 unique events
- Won 264 total medals (across all events)
- Average athlete age: 26.4 years

**Why aggregate?**
- Prediction target is team-level (total medals per country)
- Team characteristics (size, experience) are predictive
- Easier to work with ~2,000 team-year observations than 200,000+ athlete records

## Step 4: Apply Aggregation to All Teams

In [26]:
# Group by country (NOC) and year, then apply our summary function
# This creates one row per country per Olympic year
team = athletes.groupby(["NOC", "Year"]).apply(team_summary)

# Reset index to make 'team' and 'country' regular columns
team = team.reset_index(drop=True)

# Remove rows with missing values (mainly missing height/weight data)
team = team.dropna()

print(f"Team-level dataset shape: {team.shape}")
print(f"\nSample of aggregated data:")
team.head(10)

Team-level dataset shape: (2613, 9)

Sample of aggregated data:


/tmp/ipython-input-1874133367.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  team = athletes.groupby(["NOC", "Year"]).apply(team_summary)


,team,country,year,events,athletes,age,height,weight,medals
3,AFG,Afghanistan,1960,13,16,23.312500,170.687500,69.437500,0
4,AFG,Afghanistan,1964,8,8,22.000000,161.000000,64.250000,0
5,AFG,Afghanistan,1968,5,5,23.200000,170.200000,70.000000,0
6,AFG,Afghanistan,1972,8,8,29.000000,168.333333,63.750000,0
7,AFG,Afghanistan,1980,11,11,23.636364,168.363636,63.181818,0
10,AFG,Afghanistan,2004,5,5,18.600000,170.750000,64.750000,0
11,AFG,Afghanistan,2008,4,4,22.500000,179.250000,62.750000,1
12,AFG,Afghanistan,2012,6,6,24.833333,171.666667,60.833333,1
13,AFG,Afghanistan,2016,3,3,24.666667,173.666667,74.000000,0
15,AHO,Netherlands Antilles,1960,4,5,26.200000,170.200000,83.900000,0


In [27]:
# Example: Look at a specific country's progression over time
print("USA's Olympic participation over time:")
team[team["team"] == "USA"].tail(10)

USA's Olympic participation over time:


,team,country,year,events,athletes,age,height,weight,medals
2691,USA,United States,1976,189,554,23.000000,177.292948,71.009946,164
2692,USA,United States,1984,217,693,24.599132,177.182609,71.615942,352
2693,USA,United States,1988,230,715,25.213184,177.099859,71.382959,207
2694,USA,United States,1992,248,734,25.664393,176.950549,71.854824,224
2695,USA,United States,1996,263,839,26.369487,177.528327,73.171582,259
2696,USA,United States,2000,265,764,26.628272,177.325000,73.598155,242
2697,USA,United States,2004,254,726,26.517906,177.012431,73.594744,263
2698,USA,United States,2008,258,763,26.359109,177.668414,74.447090,317
2699,USA,United States,2012,245,689,26.664731,178.330914,74.692308,248
2700,USA,United States,2016,245,719,26.381085,178.041783,73.750696,264


## Step 5: Create Historical Performance Features

**Key Insight**: Past performance is highly predictive of future medals!

We create two historical features:
1. **prev_medals**: Medals won at the immediate previous Olympics
2. **prev_3_medals**: Rolling average of medals over previous 3 Olympics

### Why These Features Matter:
- **prev_medals**: Captures recent momentum (improving/declining)
- **prev_3_medals**: Captures long-term sustained performance
- Countries with strong history tend to continue performing well
- These features account for ~40% of model predictive power!

In [30]:
def prev_medals(data):
    # Ensure data is sorted chronologically
    data = data.sort_values("year", ascending=True)

    # Shift medals forward by 1 position (so 2016 gets 2012's medals)
    data["prev_medals"] = data["medals"].shift(1)

    # Calculate rolling mean over previous 3 Olympics
    # KEY FIX: Apply rolling only to the 'medals' column, not entire dataframe
    data["prev_3_medals"] = data["medals"].rolling(
        window=3,
        closed="left",
        min_periods=1
    ).mean()

    return data

In [31]:
# Apply the function to each country separately
team = team.groupby(["team"]).apply(prev_medals)

# Reset index to clean up the dataframe
team = team.reset_index(drop=True)

# Remove very old data (pre-1964) for consistency
# Focuses on modern Olympics with more complete data
team = team[team["year"] > 1960]

# Round values to 1 decimal place for cleaner output
team = team.round(1)

print(f"Final dataset shape: {team.shape}")
print(f"Years covered: {team['year'].min()} to {team['year'].max()}")

Final dataset shape: (2144, 11)
Years covered: 1964 to 2016


/tmp/ipython-input-603593097.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  team = team.groupby(["team"]).apply(prev_medals)


### Verify Historical Features with USA Example

In [32]:
# Let's verify the historical features work correctly
usa_data = team[team["team"] == "USA"][["year", "medals", "prev_medals", "prev_3_medals"]]

print("USA Olympic Medal History with Lagged Features:")
print("=" * 70)
print(usa_data.to_string(index=False))
print("\nVerification:")
print("- 1968's prev_medals (169) = 1964's medals ✓")
print("- 1972's prev_3_medals (153.3) = mean of 1964,1968,1972 medals ✓")

USA Olympic Medal History with Lagged Features:
 year  medals  prev_medals  prev_3_medals
 1964     169        125.0          127.3
 1968     166        169.0          139.0
 1972     171        166.0          153.3
 1976     164        171.0          168.7
 1984     352        164.0          167.0
 1988     207        352.0          229.0
 1992     224        207.0          241.0
 1996     259        224.0          261.0
 2000     242        259.0          230.0
 2004     263        242.0          241.7
 2008     317        263.0          254.7
 2012     248        317.0          274.0
 2016     264        248.0          276.0

Verification:
- 1968's prev_medals (169) = 1964's medals ✓
- 1972's prev_3_medals (153.3) = mean of 1964,1968,1972 medals ✓


### Understanding the Historical Features

**Example: USA in 2016**
- `medals`: 264 (medals won in 2016)
- `prev_medals`: 248 (medals won in 2012)
- `prev_3_medals`: 276.0 (average of 2004, 2008, 2012 medals)

**Interpretation:**
- USA is consistently strong (high prev_3_medals)
- Recent performance slightly below long-term average
- Model will use this pattern to predict future performance

## Step 6: Explore the Final Dataset

In [33]:
# Display first few rows
print("First 10 rows of processed data:")
print(team.head(10))

First 10 rows of processed data:
   team               country  year  events  athletes   age  height  weight  \
1   AFG           Afghanistan  1964       8         8  22.0   161.0    64.2   
2   AFG           Afghanistan  1968       5         5  23.2   170.2    70.0   
3   AFG           Afghanistan  1972       8         8  29.0   168.3    63.8   
4   AFG           Afghanistan  1980      11        11  23.6   168.4    63.2   
5   AFG           Afghanistan  2004       5         5  18.6   170.8    64.8   
6   AFG           Afghanistan  2008       4         4  22.5   179.2    62.8   
7   AFG           Afghanistan  2012       6         6  24.8   171.7    60.8   
8   AFG           Afghanistan  2016       3         3  24.7   173.7    74.0   
10  AHO  Netherlands Antilles  1964       4         4  28.5   171.2    69.4   
11  AHO  Netherlands Antilles  1968       4         5  31.0   173.2    67.8   

    medals  prev_medals  prev_3_medals  
1        0          0.0            0.0  
2        0     

In [34]:
# Basic statistics
print("Summary Statistics:")
print("=" * 70)
print(team.describe())

Summary Statistics:
              year       events     athletes          age       height  \
count  2144.000000  2144.000000  2144.000000  2144.000000  2144.000000   
mean   1994.507463    35.724813    73.693097    24.778591   173.900233   
std      15.384205    49.490270   127.907577     2.808559     5.357367   
min    1964.000000     1.000000     1.000000    17.000000   151.000000   
25%    1984.000000     6.000000     7.000000    23.275000   170.500000   
50%    1996.000000    13.000000    19.000000    24.700000   174.400000   
75%    2008.000000    44.000000    70.250000    26.100000   177.300000   
max    2016.000000   270.000000   839.000000    66.000000   193.000000   

            weight       medals  prev_medals  prev_3_medals  
count  2144.000000  2144.000000  2014.000000    2014.000000  
mean     69.271595    10.556437    10.248759       9.449901  
std       7.606507    33.028143    31.951920      28.232227  
min      43.000000     0.000000     0.000000       0.000000  
25%

In [35]:
# Check for missing values
print("Missing Values:")
print("=" * 70)
print(team.isnull().sum())
print(f"\nNote: {team['prev_medals'].isnull().sum()} countries have no previous Olympics data")
print("(These are first-time participants)")

Missing Values:
team               0
country            0
year               0
events             0
athletes           0
age                0
height             0
weight             0
medals             0
prev_medals      130
prev_3_medals    130
dtype: int64

Note: 130 countries have no previous Olympics data
(These are first-time participants)


In [36]:
# Distribution of medals
print("Medal Distribution:")
print("=" * 70)
print(f"Teams with 0 medals: {(team['medals'] == 0).sum()} ({(team['medals'] == 0).sum() / len(team) * 100:.1f}%)")
print(f"Teams with 1-10 medals: {((team['medals'] > 0) & (team['medals'] <= 10)).sum()}")
print(f"Teams with 11-50 medals: {((team['medals'] > 10) & (team['medals'] <= 50)).sum()}")
print(f"Teams with 50+ medals: {(team['medals'] > 50).sum()}")
print(f"\nMaximum medals by any team: {team['medals'].max():.0f}")
print(f"Average medals per team: {team['medals'].mean():.1f}")
print(f"Median medals per team: {team['medals'].median():.1f}")

Medal Distribution:
Teams with 0 medals: 1282 (59.8%)
Teams with 1-10 medals: 487
Teams with 11-50 medals: 239
Teams with 50+ medals: 136

Maximum medals by any team: 442
Average medals per team: 10.6
Median medals per team: 0.0


## Step 7: Export Processed Data

Save the cleaned and feature-engineered dataset for use in prediction models.

In [37]:
# Save to CSV file
team.to_csv("teams.csv", index=False)

print("✅ Data preprocessing complete!")
print("\nOutput file: teams.csv")
print(f"Shape: {team.shape}")
print(f"Columns: {list(team.columns)}")
print("\nThis file is ready for use in the medal prediction models.")

✅ Data preprocessing complete!

Output file: teams.csv
Shape: (2144, 11)
Columns: ['team', 'country', 'year', 'events', 'athletes', 'age', 'height', 'weight', 'medals', 'prev_medals', 'prev_3_medals']

This file is ready for use in the medal prediction models.
